In [21]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [35]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [36]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2018-01-02T02:00:00.000000000Z,112.720,112.751,112.666,112.688,112.683,112.694,0.085,112.793,112.582,0.211,-1,1,1,112.8141,112.7297,112.9829
12,2018-01-04T02:00:00.000000000Z,112.731,112.731,112.618,112.661,112.654,112.668,0.113,112.778,112.480,0.298,-1,1,1,112.8078,112.6886,113.0462
17,2018-01-04T22:00:00.000000000Z,112.750,112.806,112.730,112.774,112.767,112.780,0.076,112.838,112.713,0.125,1,-1,-1,112.7005,112.7505,112.6005
21,2018-01-05T14:00:00.000000000Z,113.213,113.276,113.082,113.160,113.153,113.166,0.194,113.312,113.021,0.291,-1,-1,-1,112.9919,113.1083,112.7591
27,2018-01-08T14:00:00.000000000Z,112.928,113.125,112.904,113.082,113.076,113.088,0.221,113.167,112.884,0.283,1,-1,-1,112.8557,112.9689,112.6293


In [37]:
pair = "USD_JPY"

In [38]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [39]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [40]:
df_trades["time"] = [parse(x) for x in df_trades.time] 
df_raw["time"] = [parse(x) for x in df_raw.time] 

In [41]:
df_trades["next"] = df_trades["time"].shift(-1)

In [79]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [82]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
0,2018-01-02 02:00:00+00:00,2018-01-04 02:00:00+00:00,2018-01-04 05:55:00+00:00,2018-01-02 06:00:00+00:00
1,2018-01-04 02:00:00+00:00,2018-01-04 22:00:00+00:00,2018-01-05 01:55:00+00:00,2018-01-04 06:00:00+00:00
2,2018-01-04 22:00:00+00:00,2018-01-05 14:00:00+00:00,2018-01-05 17:55:00+00:00,2018-01-05 02:00:00+00:00
3,2018-01-05 14:00:00+00:00,2018-01-08 14:00:00+00:00,2018-01-08 17:55:00+00:00,2018-01-05 18:00:00+00:00
4,2018-01-08 14:00:00+00:00,2018-01-09 18:00:00+00:00,2018-01-09 21:55:00+00:00,2018-01-08 18:00:00+00:00


In [84]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [89]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
    return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
    return 0.0

def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, start_price)
    else:
        return process_sell(start_index, TP, SL, prices, start_price)


def process_m5(m5_df, row):
    result = 0.0
    for index, price in enumerate(m5_df.mid_c.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            #print(f"Signal at {index} {price:.2f} {row.ENTRY:.2f} {row.SIGNAL:.2f}")
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, m5_df.mid_c.values, row.ENTRY)
            break
    return result

In [90]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    #print(f"{row.time} {signal_text(row.SIGNAL)} {row.ENTRY:.2f} {m5_data.shape}")
    total += process_m5(m5_data, row)
    #if index > 10:
    #    break
print(total)

85.0
